In [1]:
%display latex
from pylie import *
from pprint import pprint 
from collections.abc import Iterable
import functools
from operator import mul, sub
from pprint import pprint
from IPython.core.debugger import set_trace

In [2]:
var ("x1 x2 x3")
print (x1.__class__)
m = ((x1**2)*(x2**2)*(x3**3), (x1**3)*(x3**3), (x1**3)*x2*x3, x2*x3)

<class 'sage.symbolic.expression.Expression'>


In [3]:
print(m [0], "::", multipliers (m[0], m, (x3,x2,x1)))
print(m [1], "::", multipliers (m[1], m, (x3,x2,x1)))
print(m [2], "::", multipliers (m[2], m, (x3,x2,x1)))
print(m [3], "::", multipliers (m[3], m, (x3,x2,x1)))

x1^2*x2^2*x3^3 :: [x3, x2, x1]
x1^3*x3^3 :: [x3, x1]
x1^3*x2*x3 :: [x2, x1]
x2*x3 :: [x2]


In [4]:
m = ((2,2,3), (3,0,3), (3,1,1), (0,1,1))
print(m [0], "::", vec_multipliers (m[0], m, (2,1,0)))
print(m [1], "::", vec_multipliers (m[1], m, (2,1,0)))
print(m [2], "::", vec_multipliers (m[2], m, (2,1,0)))
print(m [3], "::", vec_multipliers (m[3], m, (2,1,0)))

(2, 2, 3) :: ([2, 1, 0], set())
(3, 0, 3) :: ([2, 0], {1})
(3, 1, 1) :: ([1, 0], {2})
(0, 1, 1) :: ([1], {0, 2})


In [5]:
ctx = Context([], [x3,x2,x1], Mgrevlex)

In [6]:
# now for completion
s1=[m[0], m[1]]
s2=[m[2], m[3]]
inter = []
for _m in s1:
    mul, nonmul = vec_multipliers (_m, m, (2,1,0))
    for nm in nonmul:
        hungi = list(_m)
        hungi[nm] += 1
        inter.append(tuple(hungi))
s1.extend(inter)
inter = []
for _m in s2:
    mul, nonmul = vec_multipliers (_m, m, (2,1,0))
    for nm in nonmul:
        hungi = list(_m)
        hungi[nm] += 1
        inter.append(tuple(hungi))
s2.extend(inter)

In [7]:
s1

[(2, 2, 3), (3, 0, 3), (3, 1, 3)]

In [8]:
ss1 = Reorder (s1,ctx, ascending = False); ss1

[(2, 2, 3), (3, 0, 3), (3, 1, 3)]

In [9]:
s2

[(3, 1, 1), (0, 1, 1), (3, 1, 2), (1, 1, 1), (0, 1, 2)]

In [10]:
ss2 = Reorder (s2,ctx, ascending = False)

In [11]:
print (ss2)

[(0, 1, 1), (0, 1, 2), (1, 1, 1), (3, 1, 1), (3, 1, 2)]


In [12]:
ctx_lex = Context ([], (x1,x2,x3))
ctx_grlex = Context ([], (x1,x2,x3),Mgrlex)
ctx_grevlex = Context ([], (x1,x2,x3),Mgrevlex)

In [13]:
m

((2, 2, 3), (3, 0, 3), (3, 1, 1), (0, 1, 1))

In [14]:
print (Reorder (m, ctx_lex))

[(0, 1, 1), (2, 2, 3), (3, 0, 3), (3, 1, 1)]


In [15]:
print (Reorder (m, ctx_grlex))

[(0, 1, 1), (2, 2, 3), (3, 0, 3), (3, 1, 1)]


In [16]:
print (Reorder (m, ctx_grevlex))

[(0, 1, 1), (2, 2, 3), (3, 0, 3), (3, 1, 1)]


In [17]:
lulu = Mgrevlex ((), (1,2,3))

In [18]:
print(lulu)

[ 1  1  1]
[ 0  0  0]
[ 0  0 -1]
[ 0 -1  0]
[-1  0  0]


In [19]:
grev = [[1,1,1], [0,0,-1],[0,-1,0]]

In [20]:
lulu = Mgrlex ((), (1,2,3))

In [21]:
print(lulu)

[1 1 1]
[0 0 0]
[1 0 0]
[0 1 0]
[0 0 1]


In [22]:
gr = [[1,1,1], [0,1,0],[1,0,0]]

In [23]:
gr

[[1, 1, 1], [0, 1, 0], [1, 0, 0]]

In [24]:
class Differential_Vector:
    def __init__ (self,e, ctx):
        self._e = self.obj = e
        if ctx._weight == Mlex:
            M = matrix.identity(len(self._e))
        elif ctx._weight == Mgrlex:
            M = matrix (len(self._e))
            for i in range(len(self._e)):
                M [0, i] = 1
            for i in range (len(self._e) - 1):
                M [i+1, i] = 1
        elif ctx._weight == Mgrevlex:
            M = matrix (len(self._e))
            for i in range(len(self._e)):
                M [0, i] = 1
            for i in range (1,len(self._e)):
                M [len(self._e)-i, i] = -1
        self.M = M
    def mycmp (self, a, b):
        a = self.M*vector(a)
        b = self.M*vector(b)
        v = [_a - _b for _a,_b in zip (a, b)]
        for _ in v:
            if _ < 0:
                return -1
            if _ > 0:
                return 1
        return 0
    def __lt__(self, other):
        return self.mycmp(self.obj, other.obj) < 0
    def __gt__(self, other):
        return self.mycmp(self.obj, other.obj) > 0
    def __eq__(self, other):
        return self.mycmp(self.obj, other.obj) == 0
    def __le__(self, other):
        return self.mycmp(self.obj, other.obj) <= 0
    def __ge__(self, other):
        return self.mycmp(self.obj, other.obj) >= 0
    def __ne__(self, other):
        return self.mycmp(self.obj, other.obj) != 0
        

In [25]:
var ('x y z t')
ctx1 = Context ([], (x,y,z,t)) 
ctx2 = Context ([], (x,y,z,t), Mgrlex)
ctx3 = Context ([], (x,y,z,t), Mgrevlex)

In [26]:
Differential_Vector ((0,0,0,0), ctx = ctx1)

In [27]:
a1 = Differential_Vector ((0,0,0,0), ctx = ctx2)
a2 = Differential_Vector ((0,1,0,0), ctx = ctx2)

In [28]:
Differential_Vector ((0,0,0,0), ctx = ctx3)

In [29]:
a1<a2

True

In [30]:
m1 = sorted(map (lambda _ : Differential_Vector(_, ctx1), ((2, 2 ,3), (3, 0, 3), (3,1,1), (0,1,1))))
print ([_._e for _ in m1])

[(0, 1, 1), (2, 2, 3), (3, 0, 3), (3, 1, 1)]


In [31]:
m2 = reversed(sorted(map (lambda _ : Differential_Vector(_, ctx2), ((2, 2 ,3), (3, 0, 3), (3,1,1), (0,1,1)))))
print ([_._e for _ in m2])

[(2, 2, 3), (3, 0, 3), (3, 1, 1), (0, 1, 1)]


In [32]:
m3 = reversed(sorted(map (lambda _ : Differential_Vector(_, ctx3), ((2, 2 ,3), (3, 0, 3), (3,1,1), (0,1,1)))))
print ([_._e for _ in m3])

[(2, 2, 3), (3, 0, 3), (3, 1, 1), (0, 1, 1)]


In [33]:
def in_class (r, mclass, M, vars):
    ''''''
    mult, nonmult = vec_multipliers (mclass, M , vars)
    return all (vec_degree (x, r) >= vec_degree (x, mclass) for x in mult)  and \
        all (vec_degree(x, r) == vec_degree (x, mclass) for x in nonmult)

M0=[]
for _s2 in sorted(s2):
    for _m in m:
        if not in_class (_s2, _m, m, (0,1,2) ):
            M0.append (_s2)
M0=list(set(M0))    

In [34]:
s2=M0

In [35]:
s2

[(3, 1, 2), (0, 1, 2), (1, 1, 1), (3, 1, 1), (0, 1, 1)]

In [65]:
def complete (l):
    while True:
        m0 = []
        for m in l :
            _, nonmult = vec_multipliers (m , l , (2,1,0))
            for nm in nonmult:
                _m = list(m)
                _m [nm] += 1
                if tuple (_m) == (3,2,3):
                    set_trace ()
                if not any (in_class (tuple(_m), v, l, (2,1,0)) for v in m):
                    m0.append (tuple(_m))
        if set(m0) == set() or set(m0) == set(l):
            return l
        l.extend (m0)
        l = reversed(sorted(map (lambda _ : Differential_Vector(_, ctx3), list(set(l)))))
        l = [_._e for _ in l]                
        print (l)
            
            

In [ ]:
complete ([(2, 2 ,3), (3, 0, 3), (3,1,1), (0,1,1)])

[(3, 1, 3), (2, 2, 3), (3, 1, 2), (3, 0, 3), (3, 1, 1), (1, 1, 1), (0, 1, 2), (0, 1, 1)]
> <ipython-input-65-6597d6772dc5>(11)complete()
      9                 if tuple (_m) == (Integer(3),Integer(2),Integer(3)):
     10                     set_trace ()
---> 11                 if not in_class (tuple(_m), m, l, (Integer(2),Integer(1),Integer(0))):
     12                     m0.append (tuple(_m))
     13         if set(m0) == set() or set(m0) == set(l):



ipdb>  pp m


(3, 1, 3)


ipdb>  c


[(3, 2, 3), (3, 1, 3), (2, 2, 3), (3, 1, 2), (3, 0, 3), (3, 1, 1), (2, 1, 1), (1, 1, 2), (0, 1, 3), (1, 1, 1), (0, 1, 2), (0, 1, 1)]
> <ipython-input-65-6597d6772dc5>(10)complete()
      8                 _m [nm] += Integer(1)
      9                 if tuple (_m) == (Integer(3),Integer(2),Integer(3)):
---> 10                     set_trace ()
     11                 if not in_class (tuple(_m), m, l, (Integer(2),Integer(1),Integer(0))):
     12                     m0.append (tuple(_m))



...

> <ipython-input-65-6597d6772dc5>(11)complete()
      9                 if tuple (_m) == (Integer(3),Integer(2),Integer(3)):
     10                     set_trace ()
---> 11                 if not in_class (tuple(_m), m, l, (Integer(2),Integer(1),Integer(0))):
     12                     m0.append (tuple(_m))
     13         if set(m0) == set() or set(m0) == set(l):



ipdb>  pp m


(2, 2, 3)


ipdb>  s


--Call--
> <ipython-input-33-c24c2aec5d3e>(1)in_class()
----> 1 def in_class (r, mclass, M, vars):
      2     ''''''
      3     mult, nonmult = vec_multipliers (mclass, M , vars)
      4     return all (vec_degree (x, r) >= vec_degree (x, mclass) for x in mult)  and         all (vec_degree(x, r) == vec_degree (x, mclass) for x in nonmult)
      5 

